In [1]:
import random
import numpy as np
import networkx as nx 
import matplotlib.pyplot as plt
from dijkstar import Graph, find_path

In [2]:
def generate_instance_matrix(nb_vertex, min_weight, max_weight, density):
  graph = np.ones((nb_vertex, nb_vertex)) * np.inf
  for i in range(nb_vertex - 1):
    for j in range(i+1, nb_vertex):
      if random.uniform(0, 1) < density:
        weight = random.randint(min_weight, max_weight)
        graph[i, j] = weight
        graph[j, i] = weight
  for i,line in enumerate(graph):
    index_zeros = np.where(line == np.inf)[0]
    index_zeros = np.delete(index_zeros, np.where(index_zeros == i))
    for j in range(len(index_zeros) - (nb_vertex - 3)):
      index = random.choice(index_zeros)
      index_zeros = np.delete(index_zeros, np.where(index_zeros == index))
      weight = random.randint(min_weight, max_weight)
      graph[i, index] = weight
      graph[index, i] = weight
  return graph

def convert_graph(graph):
  graph_dijkstra = Graph()
  for i in range(len(graph)):
    for j in range(len(graph)):
      graph_dijkstra.add_edge(i, j,graph[i][j])
  return graph_dijkstra

def get_distance_between_nodes(graph, i, j):
  if (graph[i][j] != np.inf):
    return graph[i][j], [i, j]
  else:
    if i != j:
      shortest_path = find_path(graph, i,j)
      return shortest_path.total_cost, shortest_path.nodes
    else:
      return 0, None

def get_complete_graph(graph, node_list):
  all_paths = {}
  converted_graph = convert_graph(graph)
  complete_graph = np.zeros((len(node_list), len(node_list)))
  for i in node_list:
    voisins_array = np.delete(node_list, np.where(node_list == i))
    for j in voisins_array:
      distance, path = get_distance_between_nodes(converted_graph, i, j)
      converted_graph_i = np.where(node_list == i)[0][0]
      converted_graph_j = np.where(node_list == j)[0][0]
      complete_graph[converted_graph_i][converted_graph_j] = distance
      all_paths[(converted_graph_i, converted_graph_j)] = path
  return complete_graph, all_paths

def get_random_vertex(size, nb_vertex):
  random_vertex = []
  all_vertex = np.array(range(size))
  for _ in range(nb_vertex):
    value = random.choice(all_vertex)
    random_vertex.append(value)
    all_vertex = np.delete(all_vertex, np.where(all_vertex == value))
  return np.array(random_vertex)

def generate_interval():
  interval = random.choice(range(1,4))
  shift = random.choice(range(0, 11-interval))
  return(shift*60, (shift+interval)*60)

def generate_instance(nb_vertex_matrix, min_weight, max_weight, density, random_vertex):
  graph = generate_instance_matrix(nb_vertex_matrix, min_weight, max_weight, density)
  complete_graph, all_paths = get_complete_graph(graph, random_vertex)
  return graph, complete_graph, all_paths

def get_complete_path(path, all_paths):
  complete_path = []
  for i in range(len(path) - 1):
    complete_path.extend(all_paths[(path[i], path[i+1])][:-1])
    if i == len(path) - 2:
      complete_path.append(all_paths[(path[i], path[i+1])][-1])
  return complete_path

def is_path_in_interval(path, complete_graph, interval, vertex):
  weight = 0
  for i in range(np.where(np.array(path) == vertex)[0][0]):
    weight += complete_graph[path[i]][path[i+1]]
  if weight >= interval[0] and weight <= interval[1]:
    return True
  else:
    return False

def generate_neighbours(path, complete_graph, interval, vertex):
  neighbours = []
  for i in range(len(path)):
    for j in range(i + 1, len(path)):
      new_neighbour = path.copy()
      new_neighbour[i] = path[j]
      new_neighbour[j] = path[i]
      if is_path_in_interval(new_neighbour, complete_graph, interval, vertex):
        neighbours.append(new_neighbour)
  return neighbours

def generate_random_path(nb_vertex):
  path = np.arange(nb_vertex)
  np.random.shuffle(path)
  return path

In [3]:
# variables
nb_vertex_matrix = 100
min_weight = 10
max_weight = 60
density = 0.3
nb_vertex_chosen = 6

In [4]:
#random generation of a list of vertex among all the vertex available
random_vertex = get_random_vertex(nb_vertex_matrix, nb_vertex_chosen)

# generation of the graph, the complete graph and the list of all paths
graph, complete_graph, all_paths =\
generate_instance(
  nb_vertex_matrix = nb_vertex_matrix,
  min_weight = min_weight,
  max_weight = max_weight,
  random_vertex = random_vertex,
  density = density
)

In [5]:
#generate an interval instance
interval = generate_interval()

In [6]:
#generate a random path
path = generate_random_path(nb_vertex_chosen)

In [7]:
# generate the neighbours of a path
neighbours = generate_neighbours(path, complete_graph, (0,50), 1)

In [8]:
# print("graph")
# print(graph)
print("complete_graph")
print(complete_graph)
# print("all_paths")
# print(all_paths)
# print("random_vertex")
# print(random_vertex)
# print("interval")
# print(interval)

complete_graph
[[ 0. 45. 36. 47. 30. 47.]
 [45.  0. 43. 37. 33. 34.]
 [36. 43.  0. 30. 41. 29.]
 [47. 37. 30.  0. 39. 47.]
 [30. 33. 41. 39.  0. 45.]
 [47. 34. 29. 47. 45.  0.]]


In [9]:
complete_path = get_complete_path(path, all_paths)

In [10]:
print(random_vertex)
print(path)
print(complete_path)


[96 77 54 74 48 50]
[5 0 2 1 3 4]
[50, 96, 68, 92, 54, 77, 5, 74, 70, 48]


In [11]:
from pulp import *
import numpy as np


X=[]
P = []
X_dict = {}


# variables du problème
for i in range(len(complete_graph)):
    for j in range(len(complete_graph)):
        X.append(LpVariable("X_" + str(i) + "_" + str(j), 0, 1, LpInteger))
        X_dict[("X_" + str(i) + "_" + str(j))] = LpVariable("X_" + str(i) + "_" + str(j), 0, 1, LpInteger)
        P.append(complete_graph[i][j])
# probleme
prob = LpProblem("plus court chemin", LpMinimize)

# objectif
# Somme de la multiplication des poids de chaque arc
prob += lpSum([P[i] * X[i] for i in range(len(P))])

# contraintes
# La somme d'un arc doit etre egale a 1
X_np = np.array(X)
X_np_reshape = X_np.reshape(len(complete_graph), len(complete_graph))

for i in range(len(X_np_reshape)):
    
    prob += lpSum(X_np_reshape[i]) == 1
    prob += X_np_reshape[i][i] == 0

    for j in range(len(X_np_reshape)):
        if i != j:
            prob += X_np_reshape[i][j] + X_np_reshape[j][i] <= 1
    
    

prob.solve()
print(LpStatus[prob.status])
print("Min=", value(prob.objective))

# variables resultat
for v in prob.variables():
    print("%s=%.2f"%(v.name,v.varValue), end=', ')


Optimal
Min= 192.0
X_0_0=0.00, X_0_1=0.00, X_0_2=1.00, X_0_3=0.00, X_0_4=0.00, X_0_5=0.00, X_1_0=0.00, X_1_1=0.00, X_1_2=0.00, X_1_3=0.00, X_1_4=1.00, X_1_5=0.00, X_2_0=0.00, X_2_1=0.00, X_2_2=0.00, X_2_3=0.00, X_2_4=0.00, X_2_5=1.00, X_3_0=0.00, X_3_1=0.00, X_3_2=1.00, X_3_3=0.00, X_3_4=0.00, X_3_5=0.00, X_4_0=1.00, X_4_1=0.00, X_4_2=0.00, X_4_3=0.00, X_4_4=0.00, X_4_5=0.00, X_5_0=0.00, X_5_1=1.00, X_5_2=0.00, X_5_3=0.00, X_5_4=0.00, X_5_5=0.00, 

C:\Users\avela\AppData\Roaming\Python\Python39\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


# Algo Hill-Climbing

In [12]:
def randomSolution(complete_graph):
    vertex = list(range(len(complete_graph)))
    solution = []

    for i in range(len(complete_graph)):
        randomVertex = vertex[random.randint(0, len(vertex) - 1)]
        solution.append(randomVertex)
        vertex.remove(randomVertex)

    return solution


def routeLength(complete_graph, solution):
    routeLength = 0
    for i in range(len(solution)):
        routeLength += complete_graph[solution[i - 1]][solution[i]]
    return routeLength

def getNeighbors(solution):
    neighbors = []
    for i in range(len(solution)):
        for j in range(i+1, len(solution)):
            neighbor = solution.copy()
            neighbor[i] = solution[j]
            neighbor[j] = solution[i]
            neighbors.append(neighbor)
    return neighbors

def getBestNeighbor(complete_graph, neighbours):
    bestRouteLenght = routeLength(complete_graph, neighbours[0])
    bestNeighbor = neighbours[0]
    for neighbor in neighbours:
        currentRouteLength = routeLength(complete_graph, neighbor)
        if currentRouteLength < bestRouteLenght:
            bestRouteLenght = currentRouteLength
            bestNeighbor = neighbor
    return bestNeighbor, bestRouteLenght
    

def hillClimbing(complete_graph):
    currentSolution = randomSolution(complete_graph)
    currentRouteLength = routeLength(complete_graph, currentSolution)
    neighbors = getNeighbors(currentSolution)
    bestNeighbor, bestNeighborRouteLength = getBestNeighbor(complete_graph, neighbors)

    print('liste des voisins :', neighbors)
    print('solution :', currentSolution)
    print('somme des poids de la solution :', currentRouteLength)

    while bestNeighborRouteLength < currentRouteLength:
        currentSolution = bestNeighbor
        currentRouteLength = bestNeighborRouteLength
        neighbors = getNeighbors(currentSolution)
        bestNeighbor, bestNeighborRouteLength = getBestNeighbor(complete_graph, neighbors)
        
    grpah = {
        complete_graph[i][j]: complete_graph[j][i],
        currentRouteLength: currentRouteLength
    }



    return currentSolution, currentRouteLength

    

In [13]:
def main():
    complete_graph
    print(hillClimbing(complete_graph))

if __name__ == "__main__":
    main()

    #nullos souf

liste des voisins : [[2, 5, 0, 1, 3, 4], [0, 2, 5, 1, 3, 4], [1, 2, 0, 5, 3, 4], [3, 2, 0, 1, 5, 4], [4, 2, 0, 1, 3, 5], [5, 0, 2, 1, 3, 4], [5, 1, 0, 2, 3, 4], [5, 3, 0, 1, 2, 4], [5, 4, 0, 1, 3, 2], [5, 2, 1, 0, 3, 4], [5, 2, 3, 1, 0, 4], [5, 2, 4, 1, 3, 0], [5, 2, 0, 3, 1, 4], [5, 2, 0, 4, 3, 1], [5, 2, 0, 1, 4, 3]]
solution : [5, 2, 0, 1, 3, 4]
somme des poids de la solution : 231.0
([0, 2, 5, 1, 3, 4], 205.0)
